In [13]:
import cv2 as cv
import os

In [11]:
video = cv.VideoCapture('my_video.mp4')

In [12]:
# Display relative video propreties
width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
frame_rate = video.get(cv.CAP_PROP_FPS)
frame_count = int(video.get(cv.CAP_PROP_FRAME_COUNT))
duration = frame_count / frame_rate

print("Video frame width: \t  ", width)
print("Video frame height: \t  ", height)
print("Video frame rate property:", frame_rate)
print("Video frame count: \t  ", frame_count)
print("Video duration (s): \t  ", duration)

Video frame width: 	   854
Video frame height: 	   480
Video frame rate property: 23.976023976023978
Video frame count: 	   136
Video duration (s): 	   5.672333333333333


In [4]:
frame_nb = 0
frames_left = True

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    frame_name = os.path.join('frames', f"frame_{frame_nb:04d}.jpg")
    cv.imwrite(frame_name, frame) # saves the frame
    cv.imshow('Frame', frame)     # Show frame 
    frame_nb += 1
    key = cv.waitKey(50)

print(f"{frame_nb} frames saved.")
video.release()
cv.destroyAllWindows()


136 frames saved.


In [ ]:
# Spatial Segmentation:
video = cv.VideoCapture('data/my_video.mp4')
frame_nb = 0
frames_left = True

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('grayscale_video.avi', fourcc, frame_rate, (width, height), isColor=False)

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_name = os.path.join('grayscale_frames', f"frame_{frame_nb:04d}.jpg")
    cv.imwrite(frame_name, gray) # saves the frame
    out.write(gray)  # Write the grayscale frame to the output video
    cv.imshow('Frame', gray) # Show gray frame
    frame_nb += 1
    key = cv.waitKey(50)

video.release()
out.release()
cv.destroyAllWindows()


In [6]:
frame = cv.imread('frames/frame_0089.jpg')
gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
cv.imshow('Gray Frame', gray_frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [17]:
# Algo de seuillage global heuristique
Threshold = [15 ,120, 240]

for T in Threshold:
    for i in range(5):
        print(f"Iteration {i+1}, T = {T}")
        _, binary_frame = cv.threshold(gray_frame, T, 255, cv.THRESH_BINARY)
        mean1 = gray_frame[binary_frame == 255].mean()
        mean2 = gray_frame[binary_frame == 0].mean()
        T = (mean1 + mean2) / 2
        cv.imshow('Binary Frame', binary_frame)
        cv.waitKey(0)
    cv.destroyAllWindows()

Iteration 1, T = 15
Iteration 2, T = 71.26138283342178
Iteration 3, T = 95.37107258742726
Iteration 4, T = 95.59410455848432
Iteration 5, T = 95.59410455848432
Iteration 1, T = 120
Iteration 2, T = 95.87348035114502
Iteration 3, T = 95.59410455848432
Iteration 4, T = 95.59410455848432
Iteration 5, T = 95.59410455848432
Iteration 1, T = 240
Iteration 2, T = 159.01415774412868
Iteration 3, T = 98.852059074916
Iteration 4, T = 95.62135845355246
Iteration 5, T = 95.59410455848432


In [14]:
_, binary_frame = cv.threshold(gray_frame, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
cv.imshow('original Gray Frame', gray_frame)
cv.imshow('Otsu Binary Frame', binary_frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [15]:
# Otsu segmentation on video
video = cv.VideoCapture('my_video.mp4')
frame_nb = 0
frames_left = True

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    _, binary_frame = cv.threshold(gray_frame, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    cv.imshow('Frame', frame) # Show original frame
    cv.imshow('Otsu Binary Frame', binary_frame) # Show Otsu frame 
    key = cv.waitKey(50)

video.release()
cv.destroyAllWindows()

In [10]:
video = cv.VideoCapture('grayscale_video.avi')
frames_left = True
lambda_val = 94.4257879577422 # Threshold for difference

_ , old_frame = video.read()

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    difference = cv.absdiff(old_frame, frame)
    _, difference = cv.threshold(difference, lambda_val, 255, cv.THRESH_BINARY)
    cv.imshow('Frame', frame)
    cv.imshow('Difference', difference)
    old_frame = frame
    key = cv.waitKey(50)

video.release()
cv.destroyAllWindows()

In [16]:
# Frame differencing method
folder = 'grayscale_frames'
grayscale_frames = sorted(os.listdir(folder))
lambda_val = 94.4257879577422 # Threshold for difference

N = [ 23 ] # Number of frames to consider for differencing
for n in N:
    
    for i in range(len(grayscale_frames) + n):
        if i - n < 0 or i + n >= len(grayscale_frames):
            continue
        img = cv.imread(os.path.join(folder, grayscale_frames[i]), cv.IMREAD_GRAYSCALE)
        img_after = cv.imread(os.path.join(folder, grayscale_frames[i + n]), cv.IMREAD_GRAYSCALE)
        img_before = cv.imread(os.path.join(folder, grayscale_frames[i - n]), cv.IMREAD_GRAYSCALE)
        
        diff1 = abs(img_after - img)
        _, diff1 = cv.threshold(diff1, lambda_val, 255, cv.THRESH_BINARY)
        
        diff2 = abs(img - img_before)
        _, diff2 = cv.threshold(diff2, lambda_val, 255, cv.THRESH_BINARY)
        
        result = cv.bitwise_and(diff1, diff2)
        
        cv.imshow('Result', result)
        key = cv.waitKey(50)
    cv.destroyAllWindows()


In [12]:
video = cv.VideoCapture('my_video.mp4')
frames_left = True
lambda_val = 94.4257879577422 # Threshold for difference
alpha = frame_rate / (frame_rate + 1) # Smoothing factor
_ , old_frame = video.read()
b=0

while True:
    frames_left, frame = video.read()
    if not frames_left:
        break
    diff = abs(alpha * frame + (1 - alpha) * old_frame)
    _, diff = cv.threshold(diff, lambda_val, 255, cv.THRESH_BINARY)
    cv.imshow('Frame', frame)
    cv.imshow('Difference', diff)
    old_frame = frame
    key = cv.waitKey(50)
video.release()
cv.destroyAllWindows()